In [1]:
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.pyplot import sca

from experiments.evaluation_flow.shared import GOOGLENET, RESNET_152, MOBILENET, RESNET_18, RESNET_50
from experiments.plots.util import *
import matplotlib
import matplotlib.lines as mlines

In [2]:
# define the directory containing the experiment logsfrom experiments.util.extract_data import ns2s
BASELINE = 'baseline'
PARAM_UPDATE = 'param_update'
PARAM_UPDATE_IMPROVED = 'param_update_improved'
PROVENANCE = 'provenance'

VERSION = 'version'
FINE_TUNED = 'fine-tuned'

ROOT_LOG_DIR = '/Users/nils/downloads'
NORMAL_DIR = os.path.join(ROOT_LOG_DIR, 'normal')
D_5_DIR = os.path.join(ROOT_LOG_DIR, 'dist-5')
D_10_DIR = os.path.join(ROOT_LOG_DIR, 'dist-10')
D_20_DIR = os.path.join(ROOT_LOG_DIR, 'dist-20')


U_IDS = 'u_ids'

In [3]:
def get_storage_meta(log_dir):

    # the storage consumption is determined on the server -> filter files that are server logs
    all_files = all_files_in_dir(log_dir)
    # only include files with index 0 -- otherwise it takes too long
    server_files = [f for f in all_files if 'server' in f and 'run:0' in f]

    # for all files extract the metadata (e.g. what model and dataset is used)
    server_files_and_meta = [(extract_file_meta(f), f) for f in server_files]
    # add a mapping: use_case -> model id
    server_files_and_meta = [({**f[0], **{U_IDS: use_case_ids(f[1])}}, f[1]) for f in server_files_and_meta]

    # add the total storage consumption, and the detailed storage consumption to the metadata
    # and save all metadata as a list in the variable storage_meta
    storage_meta = []
    for f in server_files_and_meta:
        use_case_id_dict = f[0][U_IDS]
        consumptions = {}
        total_consumptions = {}
        for use_case in use_case_id_dict.keys():
            s_info = storage_info(f[1], use_case_id_dict[use_case])
            consumptions[use_case] = s_info
            total_consumptions[use_case] = total_storage_consumption(s_info)

        f[0].update({CONSUMPTIONS: consumptions, TOTAL_CONSUMPTIONS: total_consumptions})
        storage_meta.append(f[0])

    return storage_meta

In [4]:
def print_cons_first_elem(storage_data):
    tmp_elem = storage_data[0]
    consumption = tmp_elem[TOTAL_CONSUMPTIONS]
    print(consumption)


In [5]:
normal_storage_meta = get_storage_meta(NORMAL_DIR)
d_5_storage_meta = get_storage_meta(D_5_DIR)
d_10_storage_meta = get_storage_meta(D_10_DIR)
d_20_storage_meta = get_storage_meta(D_20_DIR)

In [6]:
def compare_storage_consumption_approach(approach, snapshot_type=VERSION):

    normal = filter_meta(normal_storage_meta, model=MOBILENET, approach=approach, snapshot_type=snapshot_type, snapshot_dist='outdoor')
    d5 = filter_meta(d_5_storage_meta, model=MOBILENET, approach=approach, snapshot_type=snapshot_type, snapshot_dist='outdoor')
    d10 = filter_meta(d_10_storage_meta, model=MOBILENET, approach=approach, snapshot_type=snapshot_type, snapshot_dist='outdoor')
    d20 = filter_meta(d_20_storage_meta, model=MOBILENET, approach=approach, snapshot_type=snapshot_type, snapshot_dist='outdoor')

    print(F"Approach:{approach}, Snapshottype: {snapshot_type}")
    print('normal')
    print_cons_first_elem(normal)
    print('D5')
    print_cons_first_elem(d5)
    print('D10')
    print_cons_first_elem(d10)
    print('D20')
    print_cons_first_elem(d20)

In [7]:
# metas for baseline outdoor
compare_storage_consumption_approach(BASELINE)

Approach:baseline, Snapshottype: version
normal
{'U_1': 14345728, 'N0_U_3_1_1': 14345771, 'N0_U_3_1_2': 14345771, 'N0_U_3_1_3': 14345771, 'N0_U_3_1_4': 14345771, 'U_2': 14345771, 'N0_U_3_2_1': 14345771, 'N0_U_3_2_2': 14345771, 'N0_U_3_2_3': 14345771, 'N0_U_3_2_4': 14345771}
D5
{'U_1': 14345728, 'N0_U_3_1_1': 14345771, 'N1_U_3_1_1': 14345771, 'N2_U_3_1_1': 14345771, 'N3_U_3_1_1': 14345771, 'N4_U_3_1_1': 14345771, 'N0_U_3_1_2': 14345771, 'N1_U_3_1_2': 14345771, 'N2_U_3_1_2': 14345771, 'N3_U_3_1_2': 14345771, 'N4_U_3_1_2': 14345771, 'N0_U_3_1_3': 14345771, 'N1_U_3_1_3': 14345771, 'N2_U_3_1_3': 14345771, 'N3_U_3_1_3': 14345771, 'N4_U_3_1_3': 14345771, 'N0_U_3_1_4': 14345771, 'N1_U_3_1_4': 14345771, 'N2_U_3_1_4': 14345771, 'N3_U_3_1_4': 14345771, 'N4_U_3_1_4': 14345771, 'N0_U_3_1_5': 14345771, 'N1_U_3_1_5': 14345771, 'N2_U_3_1_5': 14345771, 'N3_U_3_1_5': 14345771, 'N4_U_3_1_5': 14345771, 'N0_U_3_1_6': 14345771, 'N1_U_3_1_6': 14345771, 'N2_U_3_1_6': 14345771, 'N3_U_3_1_6': 14345771, 'N4_U_3_

In [8]:
# metas for update outdoor
compare_storage_consumption_approach(PARAM_UPDATE_IMPROVED)

Approach:param_update_improved, Snapshottype: version
normal
{'U_1': 14345728, 'N0_U_3_1_1': 14327222, 'N0_U_3_1_2': 14327222, 'N0_U_3_1_3': 14327222, 'N0_U_3_1_4': 14327222, 'U_2': 14327222, 'N0_U_3_2_1': 14327222, 'N0_U_3_2_2': 14327222, 'N0_U_3_2_3': 14327222, 'N0_U_3_2_4': 14327222}
D5
{'U_1': 14345728, 'N0_U_3_1_1': 14327222, 'N1_U_3_1_1': 14327222, 'N2_U_3_1_1': 14327222, 'N3_U_3_1_1': 14327222, 'N4_U_3_1_1': 14327222, 'N0_U_3_1_2': 14327222, 'N1_U_3_1_2': 14327222, 'N2_U_3_1_2': 14327222, 'N3_U_3_1_2': 14327222, 'N4_U_3_1_2': 14327222, 'N0_U_3_1_3': 14327222, 'N1_U_3_1_3': 14327222, 'N2_U_3_1_3': 14327222, 'N3_U_3_1_3': 14327222, 'N4_U_3_1_3': 14327222, 'N0_U_3_1_4': 14327222, 'N1_U_3_1_4': 14327222, 'N2_U_3_1_4': 14327222, 'N3_U_3_1_4': 14327222, 'N4_U_3_1_4': 14327222, 'N0_U_3_1_5': 14327222, 'N1_U_3_1_5': 14327222, 'N2_U_3_1_5': 14327222, 'N3_U_3_1_5': 14327222, 'N4_U_3_1_5': 14327222, 'N0_U_3_1_6': 14327222, 'N1_U_3_1_6': 14327222, 'N2_U_3_1_6': 14327222, 'N3_U_3_1_6': 14327

In [9]:
# metas for update outdoor
compare_storage_consumption_approach(PARAM_UPDATE_IMPROVED, snapshot_type=FINE_TUNED)


Approach:param_update_improved, Snapshottype: fine-tuned
normal
{'U_1': 14345728, 'N0_U_3_1_1': 5195070, 'N0_U_3_1_2': 5195070, 'N0_U_3_1_3': 5195070, 'N0_U_3_1_4': 5195070, 'U_2': 5195070, 'N0_U_3_2_1': 5195070, 'N0_U_3_2_2': 5195070, 'N0_U_3_2_3': 5195070, 'N0_U_3_2_4': 5195070}
D5
{'U_1': 14345728, 'N0_U_3_1_1': 5195070, 'N1_U_3_1_1': 5195070, 'N2_U_3_1_1': 5195070, 'N3_U_3_1_1': 5195070, 'N4_U_3_1_1': 5195070, 'N0_U_3_1_2': 5195070, 'N1_U_3_1_2': 5195070, 'N2_U_3_1_2': 5195070, 'N3_U_3_1_2': 5195070, 'N4_U_3_1_2': 5195070, 'N0_U_3_1_3': 5195070, 'N1_U_3_1_3': 5195070, 'N2_U_3_1_3': 5195070, 'N3_U_3_1_3': 5195070, 'N4_U_3_1_3': 5195070, 'N0_U_3_1_4': 5195070, 'N1_U_3_1_4': 5195070, 'N2_U_3_1_4': 5195070, 'N3_U_3_1_4': 5195070, 'N4_U_3_1_4': 5195070, 'N0_U_3_1_5': 5195070, 'N1_U_3_1_5': 5195070, 'N2_U_3_1_5': 5195070, 'N3_U_3_1_5': 5195070, 'N4_U_3_1_5': 5195070, 'N0_U_3_1_6': 5195070, 'N1_U_3_1_6': 5195070, 'N2_U_3_1_6': 5195070, 'N3_U_3_1_6': 5195070, 'N4_U_3_1_6': 5195070, 'N0_U_3

In [10]:
# metas for prov outdoor
compare_storage_consumption_approach(PROVENANCE)


Approach:provenance, Snapshottype: version
normal
{'U_1': 14345728, 'N0_U_3_1_1': 72336625, 'N0_U_3_1_2': 72336625, 'N0_U_3_1_3': 72336625, 'N0_U_3_1_4': 72336625, 'U_2': 209511211, 'N0_U_3_2_1': 72336625, 'N0_U_3_2_2': 72336625, 'N0_U_3_2_3': 72336625, 'N0_U_3_2_4': 72336625}
D5
{'U_1': 14345728, 'N0_U_3_1_1': 72336625, 'N1_U_3_1_1': 72336625, 'N2_U_3_1_1': 72336625, 'N3_U_3_1_1': 72336625, 'N4_U_3_1_1': 72336625, 'N0_U_3_1_2': 72336625, 'N1_U_3_1_2': 72336625, 'N2_U_3_1_2': 72336625, 'N3_U_3_1_2': 72336625, 'N4_U_3_1_2': 72336625, 'N0_U_3_1_3': 72336625, 'N1_U_3_1_3': 72336625, 'N2_U_3_1_3': 72336625, 'N3_U_3_1_3': 72336625, 'N4_U_3_1_3': 72336625, 'N0_U_3_1_4': 72336625, 'N1_U_3_1_4': 72336625, 'N2_U_3_1_4': 72336625, 'N3_U_3_1_4': 72336625, 'N4_U_3_1_4': 72336625, 'N0_U_3_1_5': 72336625, 'N1_U_3_1_5': 72336625, 'N2_U_3_1_5': 72336625, 'N3_U_3_1_5': 72336625, 'N4_U_3_1_5': 72336625, 'N0_U_3_1_6': 72336625, 'N1_U_3_1_6': 72336625, 'N2_U_3_1_6': 72336625, 'N3_U_3_1_6': 72336625, 'N4_U